<a href="https://colab.research.google.com/github/jmcaussade/Arreglo-Incongruencias-Juegos-Jardin/blob/main/Incongruencias_Michel_all_bricks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install xlsxwriter
!pip install matplotlib

In [6]:
import pandas as pd
import xlsxwriter
import unicodedata
import matplotlib

def generate_colors(n):
    """Generate n distinct colors."""
    cmap = matplotlib.colormaps['tab20']
    return [matplotlib.colors.to_hex(cmap(i/n)) for i in range(n)]

def normalizar_texto(texto):
    texto = texto.lower()
    texto = unicodedata.normalize('NFD', texto).encode('ascii', 'ignore').decode("utf-8")
    return texto

###################################
##CAMBIAR NOMBRE ARCHIVO ENTRADA###
archivo_excel = 'Analisis.xlsx'

###################################
##CAMBIAR NOMBRE HOJA DE TRABAJO###
df = pd.read_excel(archivo_excel, sheet_name='Hoja1')

excluir_columnas = ["<ID>", "SKU - Chile", "Jefe de Línea", 'Descripción Web', '<Name>', "Marca", "Origen", "<Parent ID>"]

# Generate colors for initial columns
initial_col_count = len(df.columns) - len(excluir_columnas)
colores_columnas = generate_colors(initial_col_count)

for i, col in enumerate(df.columns):
    if col not in excluir_columnas:
        insert_position = df.columns.get_loc(col) + 1
        col_aparece = f"{col} aparece"
        df.insert(insert_position, col_aparece, df.apply(lambda row: normalizar_texto(str(row[col])) in
                                                        normalizar_texto(str(row['<Name>'])) or
                                                        normalizar_texto(str(row[col])) in
                                                        normalizar_texto(str(row['Descripción Web'])), axis=1))

# ##################################
# ##CALCULATE ERROR PERCENTAGE####
# ##################################

# Identify Boolean columns (those with 'aparece' in their name)
boolean_columns = [col for col in df.columns if 'aparece' in col]

# Calculate the error percentage for each row
df['Error percentage'] = df[boolean_columns].apply(lambda row: (len(boolean_columns) - row.sum()) / len(boolean_columns), axis=1)



##################################
##CAMBIAR NOMBRE ARCHIVO SALIDA###
archivo_salida = 'test_codigo1.xlsx'
##################################

writer = pd.ExcelWriter(archivo_salida, engine='xlsxwriter')
df.to_excel(writer, sheet_name='Sheet1', index=False)

workbook = writer.book
worksheet = writer.sheets['Sheet1']

format_verdadero = workbook.add_format({'bg_color': '#C6EFCE', 'font_color': '#006100'})
format_falso = workbook.add_format({'bg_color': '#FFC7CE', 'font_color': '#9C0006'})

for col_num, value in enumerate(df.columns.values):
    if value not in excluir_columnas and 'aparece' not in value:
        color_index = (col_num // 2) % len(colores_columnas)  # Cycle through colors
        cell_format = workbook.add_format({'bg_color': colores_columnas[color_index]})
        worksheet.set_column(col_num, col_num, None, cell_format)
    elif 'aparece' in value:
        # Apply conditional formatting for 'VERDADERO' (True) and 'FALSO' (False)
        worksheet.conditional_format(1, col_num, len(df), col_num,
                                     {'type': 'cell',
                                      'criteria': '=',
                                      'value': True,
                                      'format': format_verdadero})
        worksheet.conditional_format(1, col_num, len(df), col_num,
                                     {'type': 'cell',
                                      'criteria': '=',
                                      'value': False,
                                      'format': format_falso})

writer.close()
print(f"Se ha exportado el DataFrame con formato a '{archivo_salida}'.")



Se ha exportado el DataFrame con formato a 'test_codigo1.xlsx'.
